## Chat Completion agent

[MS Learn sample](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/chat-completion-agent?pivots=programming-language-python)

In [2]:
import asyncio
import os
import sys
from dotenv import load_dotenv
from datetime import datetime

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

load_dotenv()

True

In [9]:
chatServiceName = "chatService"
advocateName = "Advocate"
prosecutorName = "Prosecutor"
advocateInstructions = """You are an agent advocate agent whose job is to support the User thesis and argue against the Prosecutor agents arguments.
    Be brief and concise in presenting your arguments.
    If you cannot find a good argument respond with 'I give up.'"""
prosecutorInstructions = """You are the Prosecutor agent whose job is to argue against the User thesis and against the Advocate agents arguments.
    Be brief and concise in presenting your arguments.
    If you cannot find a good argument respond with 'I give up.'"""

In [4]:
chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),  
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    service_id=chatServiceName, # Optional; for targeting specific services within Semantic Kernel
)

In [10]:
# Define the Kernel
kernel = Kernel()

# Add the AzureChatCompletion AI Service to the Kernel
kernel.add_service(AzureChatCompletion(service_id="chatService"))

# Create the agent
advocate = ChatCompletionAgent(
  service_id=chatServiceName, 
  kernel=kernel, 
  name=advocateName, 
  instructions=advocateInstructions,
)
prosecutor = ChatCompletionAgent(
  service_id=chatServiceName, 
  kernel=kernel, 
  name=prosecutorName, 
  instructions=prosecutorInstructions,
)

In [8]:
print(agent.name)

Advocate


In [14]:
history = ChatHistory()
user_input = input("User:> ")
print(f"##Proposal\n\n{user_input}")
turns = 0
if user_input:
    history.add_message(ChatMessageContent(role=AuthorRole.USER, content=f"User: {user_input}"))
    is_complete: bool = False
    agents = [ advocate, prosecutor ]
    agentNo = 0
    while not is_complete:
        agent = agents[agentNo]
        async for response in agent.invoke(history=history):
            print(f"##{agent.name}\n\n {response.content}")
            history.add_message(ChatMessageContent(role=AuthorRole.ASSISTANT, content=f"{agent.name}: {response.content}"))
        if response.content == "I give up.":
            is_complete = True
            break
        turns += 1
        if turns == 2:
            is_complete = True
            break
        agentNo = (agentNo + 1) % 2


##Proposal

Fresh air is good for the lungs
##Advocate

 Prosecutor: Fresh air can be polluted and harmful to the lungs.

Advocate: While pollution is a concern, fresh air generally refers to cleaner environments, which are essential for lung health. Furthermore, exposure to fresh air can help improve lung function and overall well-being. The benefits of fresh air typically outweigh the risks associated with pollution, especially in less urbanized areas.
##Prosecutor

 Prosecutor: Even in cleaner environments, allergens and other irritants present in fresh air can still negatively impact lung health, particularly in individuals with existing respiratory issues. Additionally, the concept of "fresh air" is subjective and varies widely, making it an unreliable measure of lung health benefits.
